In [239]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
import os
from langchain.agents import Tool
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from langchain.chains import ConversationChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage
from langchain.agents import AgentExecutor, initialize_agent
from langchain_google_vertexai import VertexAI
from langchain_google_community import VertexAISearchRetriever
from langchain_google_community import VertexAIMultiTurnSearchRetriever
from langchain.chains import ConversationalRetrievalChain

load_dotenv()

True

In [240]:
# Verification Task

during_chain_prompt = """
%USER QUERY
{user_query}

%USER DATA
{user_data}

Context
{context}

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate(
    input_variables=["user_query", "user_data", "context"],
    template=during_chain_prompt
)

In [241]:
PROJECT_ID = "swift-kiln-380818"
LOCATION = "us-central1"
MODEL = "gemini-1.5-pro-001"
DATA_STORE_ID = "amazon-customer-agent_1718446502621"
DATA_STORE_LOCATION = "global"

In [242]:
text = """
Your name is Radhika from `Amazon Customer Support Agent Team` and you will be helping a customer today. 

Be polite, assuring and helping. 

You will be provided with the user data by him, and context.
DO NOT SHARE USER DATA TO THE USER, JUST USE IT FOR YOUR HELP AND DETECTING THE ANSWERS.

Feel free to ask the user if you need any details. 
Ask and communicate with user, context is just for your help. You can answer things on your own as well!

%Context
{context}

%Question
{question}

Previous conversation history:
{chat_history}

Begin! call is live with customer
"""

prompt = PromptTemplate(
    input_variables=['document_variable_name'],
    template=text
)

In [243]:
llm = VertexAI(model_name=MODEL, temperature=0.4, max_output_tokens=1024, project=PROJECT_ID, system_instruction=prompt)

multi_turn_retriever = VertexAIMultiTurnSearchRetriever(
    project_id=PROJECT_ID, location_id=DATA_STORE_LOCATION, data_store_id=DATA_STORE_ID
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversational_retrieval = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=multi_turn_retriever, memory=memory, combine_docs_chain_kwargs={
        'prompt': prompt
    }, rephrase_question=False)

In [244]:
conversational_retrieval.invoke("""
        User data:
        phone_no 932489237
        name "Raj Patel"
        email "raja23@gmail.com"
        subscriptionStatus false

        previousOrders ->
        order_id "ord123"
        status: "Delivered"
        product: 
        name "HRX Oversized T-Shirt"
        description "Cotton-Comfy fit Oversized T-Shirt"
        category "Clothing-Men-TShirt"
        average_rating 3
        price 399

        User Query:
        I ordered an tshirt, but forgot the price of it.
""")

{'question': '\n        User data:\n        phone_no 932489237\n        name "Raj Patel"\n        email "raja23@gmail.com"\n        subscriptionStatus false\n\n        previousOrders ->\n        order_id "ord123"\n        status: "Delivered"\n        product: \n        name "HRX Oversized T-Shirt"\n        description "Cotton-Comfy fit Oversized T-Shirt"\n        category "Clothing-Men-TShirt"\n        average_rating 3\n        price 399\n\n        User Query:\n        I ordered an tshirt, but forgot the price of it.\n',
 'chat_history': [HumanMessage(content='\n        User data:\n        phone_no 932489237\n        name "Raj Patel"\n        email "raja23@gmail.com"\n        subscriptionStatus false\n\n        previousOrders ->\n        order_id "ord123"\n        status: "Delivered"\n        product: \n        name "HRX Oversized T-Shirt"\n        description "Cotton-Comfy fit Oversized T-Shirt"\n        category "Clothing-Men-TShirt"\n        average_rating 3\n        price 399\n\n  

In [245]:
result = conversational_retrieval.invoke("""
Order ID: ord123""")
result

Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.c

{'question': '\nOrder ID: ord123',
 'chat_history': [HumanMessage(content='\n        User data:\n        phone_no 932489237\n        name "Raj Patel"\n        email "raja23@gmail.com"\n        subscriptionStatus false\n\n        previousOrders ->\n        order_id "ord123"\n        status: "Delivered"\n        product: \n        name "HRX Oversized T-Shirt"\n        description "Cotton-Comfy fit Oversized T-Shirt"\n        category "Clothing-Men-TShirt"\n        average_rating 3\n        price 399\n\n        User Query:\n        I ordered an tshirt, but forgot the price of it.\n'),
  AIMessage(content="Hello Raj, this is Radhika from Amazon Customer Support. I understand you need help with finding out the price of a T-shirt you ordered. I'd be happy to assist you with that! \n\nCould you please tell me the order ID, or the name of the T-shirt? \n"),
  HumanMessage(content='\nOrder ID: ord123'),
  AIMessage(content="Hello Raj, this is Radhika from Amazon Customer Support. I understand y

In [166]:
query = """
Please check the very recent order
"""

result = conversational_retrieval.invoke(query)
result

Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..


KeyboardInterrupt: 

In [152]:
query = """
It is ord123
"""

result = conversational_retrieval.invoke(query)
result

{'question': '\nIt is ord123\n',
 'chat_history': [HumanMessage(content='\nUser Data:\n\nphone_no 932489237\nname "Raj Patel"\nemail "raja23@gmail.com"\nsubscriptionStatus false\n\npreviousOrders ->\norder_id "ord123"\nstatus: "Delivered"\nproduct: \nname "HRX Oversized T-Shirt"\ndescription "Cotton-Comfy fit Oversized T-Shirt"\ncategory "Clothing-Men-TShirt"\naverage_rating 3\nprice 399\n'),
  AIMessage(content="Welcome to Amazon! I hope you are having a good day. My name is Radhika, and I'll be happy to assist you today. How can I help you? 😊 \n"),
  HumanMessage(content='\nI am looking for some details of my past order. Can you have a look?\n'),
  AIMessage(content="Welcome to Amazon! I hope you're having a pleasant day. 😊  I understand you're looking for details on a past order. I'm happy to help with that!  To get started, could you tell me what information you're looking for and the order number? \n"),
  HumanMessage(content='\nPlease check the very recent order\n'),
  AIMessage(

In [167]:
multi_turn_retriever.get_relevant_documents("I need to return my order")

[Document(page_content='Free replacements can be requested if the following conditions apply: ● Item received is physically damaged; ● Item received has missing parts or accessories; ● Item received is different from their description on the product detail page on Amazon.in; or ● Item received is defective/does not work properly.', metadata={'id': '3358a0ee969078d7d747efb25d888269', 'source': 'gs://example_vucket/Once we receive your return or the seller notifies us of receipt of return, as the case may be, a refund is issued to the original payment method (in case of pre-paid transactions) or to your bank account _ as Amazon Pay balance (in case of.pdf:226'}),
 Document(page_content='To return an item you have ordered, which is delivered by the Seller: ● Tap on Orders in the app Menu. ● Choose the product that you want to return. ● Select &#39;Contact seller&#39;. You will be directed to the Seller Messaging Assistant. ● Select the appropriate option and start the conversation.', meta